In [156]:
import pandas as pd
from pathlib import Path
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif, SelectKBest, chi2, SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from skfeature.function.similarity_based import fisher_score


In [157]:
starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_model/Resources/BankChurnersPrimary.csv"))
starter_df.sample(5)

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
8087,718825908,Existing Customer,60,F,1,Doctorate,Single,Unknown,Blue,47,...,3803.0,1448,2355.0,0.565,4575,78,0.592,0.381,0.000045,0.99995
5082,710854008,Existing Customer,48,M,3,Unknown,Unknown,$40K - $60K,Blue,37,...,6541.0,1119,5422.0,0.496,3902,63,0.432,0.171,0.000372,0.99963
6910,804413508,Existing Customer,41,M,1,High School,Single,$80K - $120K,Blue,36,...,12084.0,1638,10446.0,0.695,4506,90,0.957,0.136,0.000175,0.99983
2507,716400633,Existing Customer,33,M,3,High School,Single,$40K - $60K,Blue,24,...,5156.0,1427,3729.0,0.480,2057,54,0.421,0.277,0.000305,0.99969
8861,717895833,Existing Customer,55,M,3,College,Single,$120K +,Blue,44,...,6455.0,1837,4618.0,0.717,8001,99,0.707,0.285,0.000193,0.99981


In [158]:
starter_df.drop(columns=[
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
'CLIENTNUM',
'Education_Level', 
'Marital_Status', 
'Income_Category'
], 
inplace= True)

In [159]:

enc = OneHotEncoder(sparse=False)
categorical_variables = ['Card_Category']
encoded_data = enc.fit_transform(starter_df[categorical_variables])
encoded_dataframe = pd.DataFrame(encoded_data, columns = enc.get_feature_names(categorical_variables))
starter_df.drop(columns=['Card_Category'], inplace=True)
starter_df = pd.concat([starter_df, encoded_dataframe.set_axis(starter_df.index)], axis=1)
starter_df.sample(10)

c:\python_files\Anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Attrition_Flag,Customer_Age,Gender,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Card_Category_Blue,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
9866,Existing Customer,34,F,2,23,4,2,2,7594.0,1442,6152.0,0.753,15026,99,0.678,0.190,1.0,0.0,0.0,0.0
9132,Existing Customer,52,F,2,36,1,2,2,2540.0,1093,1447.0,0.783,9339,99,0.737,0.430,1.0,0.0,0.0,0.0
1311,Attrited Customer,40,M,4,22,3,1,3,34516.0,0,34516.0,0.640,1028,26,0.300,0.000,0.0,0.0,0.0,1.0
1740,Existing Customer,56,M,2,41,3,2,2,5887.0,659,5228.0,0.650,1193,26,0.300,0.112,1.0,0.0,0.0,0.0
3492,Attrited Customer,40,M,4,29,3,3,2,12544.0,1636,10908.0,0.503,1758,51,0.457,0.130,1.0,0.0,0.0,0.0
7603,Existing Customer,44,F,3,27,5,1,1,7751.0,1965,5786.0,0.782,4543,90,0.731,0.254,1.0,0.0,0.0,0.0
5531,Existing Customer,46,F,4,26,6,1,3,1859.0,1379,480.0,0.836,4260,72,0.714,0.742,1.0,0.0,0.0,0.0
2168,Existing Customer,39,F,2,26,5,1,2,2575.0,1424,1151.0,0.543,1487,33,0.737,0.553,1.0,0.0,0.0,0.0
9515,Existing Customer,46,M,4,33,1,2,1,13490.0,1567,11923.0,0.795,13159,105,0.750,0.116,1.0,0.0,0.0,0.0
3069,Existing Customer,45,M,3,38,3,1,2,23973.0,0,23973.0,0.779,3560,71,0.511,0.000,1.0,0.0,0.0,0.0


In [160]:
starter_df = starter_df.replace({'Gender' : {
'M': 0, 
'F': 1
}})
starter_df.Gender = starter_df.Gender.astype(int)

In [161]:
starter_df = starter_df.replace({'Attrition_Flag' : {
'Attrited Customer': 0, 
'Existing Customer': 1
}})
starter_df.Attrition_Flag = starter_df.Attrition_Flag.astype(int)

In [162]:
y = starter_df['Attrition_Flag']
X = starter_df.drop(columns=['Attrition_Flag'])

In [163]:
gbc = GradientBoostingClassifier()
sfs = SequentialFeatureSelector(gbc, n_features_to_select=8, direction='forward')
sfs.fit(X, y)
X_selection = sfs.transform(X)

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X_selection, y, random_state=1)

In [165]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [166]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [167]:
gbc_model = GradientBoostingClassifier(
n_estimators=500,
random_state= 2
)

In [168]:
gbc_model.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=500, random_state=2)

In [169]:
gbc_test_predictions = gbc_model.predict(X_test)

In [170]:
accuracy = accuracy_score(y_test, gbc_test_predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 96.48%


In [171]:
gbc_test_matrix = confusion_matrix(y_test, gbc_test_predictions)
print(gbc_test_matrix)

[[ 372   33]
 [  56 2071]]


In [172]:
gbc_testing_report = classification_report(y_test, gbc_test_predictions)
print(gbc_testing_report)

              precision    recall  f1-score   support

           0       0.87      0.92      0.89       405
           1       0.98      0.97      0.98      2127

    accuracy                           0.96      2532
   macro avg       0.93      0.95      0.94      2532
weighted avg       0.97      0.96      0.97      2532



In [173]:
imb_gbc_testing_report = classification_report_imbalanced(y_test, gbc_test_predictions)
print(imb_gbc_testing_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      0.92      0.97      0.89      0.95      0.89       405
          1       0.98      0.97      0.92      0.98      0.95      0.90      2127

avg / total       0.97      0.96      0.93      0.97      0.95      0.90      2532



In [174]:
sfs.get_support().tolist()

[True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False]

In [175]:
results = pd.DataFrame(sfs.get_support())
features = X.columns.values
features = pd.DataFrame(features)

results_df = pd.concat([features, results.set_axis(features.index)], axis=1)
results_df = results_df[results_df != False].dropna()
results_df

,0,0
0,Customer_Age,True
4,Total_Relationship_Count,True
5,Months_Inactive_12_mon,True
8,Total_Revolving_Bal,True
10,Total_Amt_Chng_Q4_Q1,True
11,Total_Trans_Amt,True
12,Total_Trans_Ct,True
13,Total_Ct_Chng_Q4_Q1,True


### If 8 features were picked:
Customer_Age, Total_Relationship_Count, Months_Inactive_12_mon, Total_Revolving_Bal, Total_Amt_Chng_Q4_Q1, Total_Trans_Amt, Total_Trans_Ct, Total_Ct_Chng_Q4_Q1
